In [1]:
# Include project root path into PYTHONPATH

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import pandas as pd
import matplotlib.pyplot as plt
from os import walk
import numpy as np
from tqdm import tqdm
from astropy.time import Time
from os.path import join
from matplotlib.patches import Wedge, Rectangle
from src.cmesrc.config import SWAN_DATA_DIR, ALL_MATCHING_HARPS_DATABASE_PICKLE, DIMMINGS_MATCHED_TO_HARPS_PICKLE, OVERVIEW_FIGURES_DIR, FLARES_MATCHED_TO_HARPS_PICKLE, MAIN_DATABASE_PICKLE
from src.cmesrc.utils import get_closest_harps_timestamp
from src.harps.harps import Harps
from src.cmes.cmes import CME
from src.flares.flares import Flare
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.coordinates import Angle
from sunpy.coordinates import frames
from src.dimmings.dimmings import Dimming
import sunpy.map
from src.scripts.collect_results.collate_results import MAX_DIMMING_TIME_AFTER_CME, MAX_FLARE_TIME_AFTER_CME, MAX_DIMMING_TIME_BEFORE_CME, MAX_FLARE_TIME_BEFORE_CME

transparent_white = (1, 1, 1, 0.4)


In [7]:

def plot_cme(ax, sunpy_map, principal_angle, angular_width, halo = False, distance=0.95, npoints=100, linestyle="solid", color="green", linewidth=1, alpha=1):
    center = sunpy_map.world_to_pixel(sunpy_map.center)
    rad = min(center) * distance

    if halo:
        principal_angle = Angle('0d')
        angular_width = Angle('359.99d')
    else:
        principal_angle = Angle(f'{principal_angle}d')
        angular_width = Angle(f'{angular_width}d')

    ninety_deg = Angle('90d')

    lower_angle = (principal_angle + ninety_deg - angular_width / 2).to(u.rad)
    upper_angle = (principal_angle + ninety_deg + angular_width / 2).to(u.rad)

    angles = np.linspace(lower_angle, upper_angle, npoints)

    modified_x = []
    modified_y = []

    for angle in angles:
        modified_x.append((rad * np.cos(angle)).value) 
        modified_y.append((rad * np.sin(angle)).value)

    if not halo:
        modified_x.insert(0,0)
        modified_y.insert(0,0)
        modified_x.append(0)
        modified_y.append(0)

    modified_x = np.array(modified_x) * u.pix
    modified_y = np.array(modified_y) * u.pix

    real_x = modified_x + center[0]
    real_y = modified_y + center[1]

    points = SkyCoord(np.array(list(map(sunpy_map.pixel_to_world, real_x, real_y))))

    ax.plot_coord(points, c=color, linewidth=linewidth, linestyle=linestyle, label=f"PA={principal_angle}, WIDTH={angular_width}", zorder=20, alpha=alpha)
    return ax

In [8]:
def generate_blank_map():
    observer = frames.HeliographicStonyhurst(0 * u.rad, 0 * u.rad, radius= 1 * u.AU)

    header_data = np.full((10, 10), np.nan)

    ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec, obstime=cme_time,
                    observer=observer, frame=frames.Helioprojective)

    header = sunpy.map.make_fitswcs_header(header_data, ref_coord, scale=[220, 220]*u.arcsec/u.pixel)

    blank_map = sunpy.map.Map(header_data, header)
    return blank_map

In [9]:

main_data = pd.read_pickle(MAIN_DATABASE_PICKLE)

main_data["CME_HARPNUM_MATCH"] = (
    main_data["HARPS_SPAT_CONSIST"] &
    main_data["DIMMING_FLAG"] &
    main_data["FLARE_FLAG"] &
    main_data["FLARE_CLASS_FLAG"]
)

harps_data = pd.read_pickle(MAIN_DATABASE_PICKLE)

harps_data["CME_HARPNUM_MATCH"] = (
    harps_data["HARPS_SPAT_CONSIST"] &
    harps_data["DIMMING_FLAG"] &
    harps_data["FLARE_FLAG"] &
    harps_data["FLARE_CLASS_FLAG"]
)

harps_data.set_index("CME_HARPNUM_ID", drop=True, inplace=True)
dimmings_data = pd.read_pickle(DIMMINGS_MATCHED_TO_HARPS_PICKLE)
#dimmings_data.set_index("CME_ID", drop=True, inplace=True)

flares_data = pd.read_pickle(FLARES_MATCHED_TO_HARPS_PICKLE)
#flares_data.set_index("CME_ID", drop=True, inplace=True)

grouped_main_data = main_data.groupby("CME_ID")

grouped_harps_data = harps_data.groupby("CME_ID")
#grouped_dimmings_data = dimmings_data.groupby("CME_ID")
#grouped_flares_data = flares_data.groupby("CME_ID")

grouped_dimmings_data = dimmings_data.groupby("dimming_id")

new_dimming_rows = []

for dimming_id, dimming_data in grouped_dimmings_data:
    sorted_dimming_data = dimming_data.sort_values(by="MATCH", ascending=False)
    choosen_dimming = sorted_dimming_data.iloc[0]
    is_matched = choosen_dimming["MATCH"]

    new_row = {
        "DIMMING_ID": dimming_id,
        "DIMMING_DATE": choosen_dimming["start_time"],
        "DIMMING_LON": choosen_dimming["longitude"],
        "DIMMING_LAT": choosen_dimming["latitude"],
        "MATCH": is_matched,
        "HARPNUM": choosen_dimming["HARPNUM"] if is_matched else None,
    }

    new_dimming_rows.append(new_row)

clean_dimming_data = pd.DataFrame(new_dimming_rows)

In [10]:

#for cme_id in tqdm(np.random.choice(list(grouped_harps_data.groups.keys()), N, replace=False)):
#for cme_id in tqdm(list(grouped_harps_data.groups.keys())):
cme_ids = list(grouped_harps_data.groups.keys())

plot_cme_ids = [cme_id for cme_id in cme_ids if "201404" in cme_id]
#for cme_id in tqdm(list(grouped_harps_data.groups.keys())):
for cme_id in tqdm(plot_cme_ids):

    harps_rows = grouped_harps_data.get_group(cme_id)

    cme_time = harps_rows["CME_DATE"].to_list()[0]
    cme_pa = harps_rows["CME_PA"].to_list()[0]
    cme_width = harps_rows["CME_WIDTH"].to_list()[0]
    cme_halo = harps_rows["CME_HALO"].to_list()[0]

    dimming_times = clean_dimming_data["DIMMING_DATE"].to_list()
    time_mask = (
        (dimming_times > cme_time - MAX_DIMMING_TIME_BEFORE_CME) &
        (dimming_times < cme_time + MAX_DIMMING_TIME_AFTER_CME)
    )
    dimming_rows = clean_dimming_data[time_mask]

    flare_times = flares_data["FLARE_DATE"].to_list()
    time_mask = (
        (flare_times > cme_time - MAX_FLARE_TIME_BEFORE_CME) &
        (flare_times < cme_time + MAX_FLARE_TIME_AFTER_CME)
    )
    flare_rows = flares_data[time_mask]

    if ((len(dimming_rows) == 0) and (len(flare_rows) == 0)):
        continue

    cme = CME(cme_time, cme_pa, cme_width, halo=cme_halo)

    blank_map = generate_blank_map()

    fig = plt.figure(figsize=(8,8), dpi=200)
    ax = fig.add_subplot(projection=blank_map)

    blank_map.plot(axes=ax)
    blank_map.draw_limb(axes=ax, color="k")
    blank_map.draw_grid(axes=ax, color="k", zorder=-1, alpha=0.2)
#
    plot_cme(ax, blank_map, cme_pa, cme_width, distance=1.1, npoints=100, halo=cme_halo)

    if not cme_halo:
        plot_cme(ax, blank_map, cme_pa, cme_width + cme.WIDTH_EXTRA_ANGLE, distance=1.1, npoints=100, linewidth=1, linestyle="dotted", alpha=0.5)


    plotted_harps_dict = dict()

    for idx, harps_data_row in harps_rows.iterrows():

        harps_data_values = harps_data_row[["CME_DATE","HARPS_LONDTMIN","HARPS_LATDTMIN","HARPS_LONDTMAX","HARPS_LATDTMAX"]]

        harpsnum = harps_data_row["HARPNUM"]

        harps = Harps(*harps_data_values)
        rotated_harps = harps#.rotate_bbox(cme_time)

        plotted_harps_dict[harpsnum] = rotated_harps

        if harps_data_row["CME_HARPNUM_MATCH"]:
            color = "#1BBF7B"
        elif harps_data_row["HARPS_SPAT_CONSIST"]:
            color = "#28C0D7"
        else:
            color = "#D73F28"

        blank_map.draw_quadrangle(
                bottom_left=rotated_harps.LOWER_LEFT.get_skycoord(),
                top_right=rotated_harps.UPPER_RIGHT.get_skycoord(),
                axes=ax,
                zorder=10,
                edgecolor=color,
                )

        ax.annotate(harpsnum, 
                    (rotated_harps.get_centre_point().LON, rotated_harps.get_centre_point().LAT),
                    xytext=(rotated_harps.get_centre_point().LON, rotated_harps.LOWER_LEFT.LAT-5),
                    xycoords=ax.get_transform('heliographic_stonyhurst'),
                    backgroundcolor=transparent_white,
                    color='k',
                    horizontalalignment='center', 
                    verticalalignment='top',
                    fontsize=6,
                    zorder=30
                    )


        ax.plot_coord(rotated_harps.get_centre_point().get_skycoord(), c="k", zorder=10, marker=".", markersize=1)

    if len(dimming_rows) > 0:
        for dimming_id, dimming_data in dimming_rows.iterrows():

            HARPNUM = dimming_data["HARPNUM"]

            if dimming_data["MATCH"]:
                color = "#28C0D7"
            else:
                color = "#D73F28"

            dimming = Dimming(
                    date = dimming_data["DIMMING_DATE"],
                    lon = dimming_data["DIMMING_LON"],
                    lat = dimming_data["DIMMING_LAT"]
                    )

            dimming_skycoord = dimming.point.rotate_coords(cme_time).get_skycoord()

            ax.plot_coord(dimming_skycoord, c=color, zorder=10, marker="X", markersize=10, markeredgecolor="k")

            ax.annotate(f"D{dimming_id}", 
                        (0, 0),
                        xytext=(dimming.point.LON, dimming.point.LAT + 5),
                        xycoords=ax.get_transform('heliographic_stonyhurst'),
                        backgroundcolor=transparent_white,
                        color='k',
                        horizontalalignment='center', 
                        verticalalignment='top',
                        fontsize=6,
                        zorder=10
                        )

                
            if dimming_data["MATCH"]:
                harps_centre = plotted_harps_dict[dimming_data["HARPNUM"]].get_centre_point().get_skycoord()
                line = SkyCoord([dimming_skycoord, harps_centre])
                ax.plot_coord(line, c="k", linewidth=1, zorder=5)


    if len(flare_rows) > 1:
        for flare_id, flare_data in flare_rows.iterrows():

            color = "#28C0D7"

            flare_class = flare_data["FLARE_CLASS"]

            flare = Flare(
                    date = flare_data["FLARE_DATE"],
                    lon = flare_data["FLARE_LON"],
                    lat = flare_data["FLARE_LAT"]
                    )

            flare_skycoord = flare.point.rotate_coords(cme_time).get_skycoord()

            ax.plot_coord(flare_skycoord, c=color, zorder=10, marker="*", markersize=10, markeredgecolor="k")

            ax.annotate(f"F{flare_id}-{flare_class}", 
                        (0, 0),
                        xytext=(flare.point.LON, flare.point.LAT + 5),
                        xycoords=ax.get_transform('heliographic_stonyhurst'),
                        backgroundcolor=transparent_white,
                        color='k',
                        horizontalalignment='center', 
                        verticalalignment='top',
                        fontsize=6,
                        zorder=10
                        )
                
            harps_centre = plotted_harps_dict[flare_data["HARPNUM"]].get_centre_point().get_skycoord()
            line = SkyCoord([flare_skycoord, harps_centre])
            ax.plot_coord(line, c="k", linewidth=1, zorder=5)

    plt.savefig(OVERVIEW_FIGURES_DIR + str(cme_id) + ".png", dpi=100, facecolor="#555555")
    plt.savefig(OVERVIEW_FIGURES_DIR + str(cme_id) + ".svg", bbox_inches="tight")
    plt.clf()
    plt.cla()
    plt.close()


100%|██████████| 68/68 [03:29<00:00,  3.08s/it]
